In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from datetime import datetime
import os
import geopandas as gpd
import keras
import gee
import utils


In [3]:
region_name = 'pang_war_myanmar'
#region_name = 'test_region_small'


tile_size = 576 # this is the around the max size that GEE exports allow with 12-band imagery
tile_padding = 24

start_date = datetime(2024, 1, 1)
end_date = datetime(2025, 12, 31)
clear_threshold = 0.6

pred_threshold = 0.8

region = gpd.read_file(f'../data/boundaries/{region_name}.geojson').geometry[0].__geo_interface__
tiles = utils.create_tiles(region, tile_size, tile_padding)
print(f"Created {len(tiles):,} tiles")

Created 72 tiles


In [4]:
# load an ensembled model

#connect to cpu only
#os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

model_name = '48px_v4.0_ensemble_2024-06-14'
model = keras.models.load_model(f'../models/{model_name}.h5', compile=False)

2024-06-17 10:19:54.120642: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2024-06-17 10:19:54.120659: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2024-06-17 10:19:54.120665: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2024-06-17 10:19:54.120695: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-06-17 10:19:54.120710: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [5]:
import gee
data_pipeline = gee.Data_Extractor(
    tiles, 
    start_date, 
    end_date, 
    clear_threshold, 
    sensor='NICFI'
    )

In [6]:
predictions=data_pipeline.make_predictions(model, pred_threshold=pred_threshold, tries=2)


 12%|█▎        | 1/8 [00:11<01:22, 11.80s/it]

Found 15 positives.


 25%|██▌       | 2/8 [00:23<01:10, 11.73s/it]

Found 70 positives.


 38%|███▊      | 3/8 [00:35<00:58, 11.72s/it]

...

 88%|████████▊ | 7/8 [01:24<00:12, 12.26s/it]

Found 161 positives.


100%|██████████| 8/8 [01:33<00:00, 11.74s/it]


Found 162 positives.


0it [00:00, ?it/s]


In [9]:
# write the predictions to a file
print(len(predictions), 'chips with predictions above', pred_threshold)
# write the predictions to a file
model_version_name = '_'.join(model_name.split('_')[0:2])
# if the outputs directory does not exist, create it
if not os.path.exists(f'../data/outputs/{model_version_name}'):
    os.makedirs(f'../data/outputs/{model_version_name}')
time_period = f"{start_date.month}_{start_date.year}-{end_date.month}_{end_date.year}"
predictions.to_file(f"../data/outputs/{model_version_name}/{region_name}_{model_version_name}_{pred_threshold:.2f}_{time_period}.geojson", driver="GeoJSON")

162 chips with predictions above 0.8


In [10]:
import plotly.express as px

predictions['lat'] = predictions['geometry'].apply(lambda x: x.centroid.y)
predictions['lon'] = predictions['geometry'].apply(lambda x: x.centroid.x)

fig = px.scatter_mapbox(predictions, lat="lat", lon="lon", zoom=10)
fig.update_layout(mapbox_style="satellite-streets", mapbox_accesstoken='pk.eyJ1Ijoib2JhbGxpbmdlciIsImEiOiJjazdvbXFkMWEwYXVrM2ZtdHU0cTd0cG5hIn0.vIj9s8F6M49ZP5DAh9Kd3Q')
fig.show()